# Feature Extraction

In [1]:
import pandas as pd
import re
from sklearn import preprocessing


def cleanData(text):
    removeList = ["@handle","@handle:"]
    text_r = " ".join([char for char in text.split(' ') if char not in removeList and not any(i.isdigit() for i in char) and char.isdigit]) 
    return text_r

def removeHyperLink(text):
    text,count = re.subn(r'http?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    return(text,count)
    
def removeSpChar(text):
    new = re.sub('[@_!#$%^&*()<>?/\|}{~:\']' ,'', text)
    count = len(text) - len(new)
    return new,count

def removeRT(text):
    text_r = " ".join([char for char in text.split(' ') if char != 'RT'])
    return text_r


##Normalization

# def normalizeDF(df_orig):
#     normalizeColumn(df_orig,'chCount')
#     normalizeColumn(df_orig,'wdCount')
#     normalizeColumn(df_orig,'spCount')
#     #normalizeColumn(df_orig,'isCap')
#     normalizeColumn(df_orig,'hashtags')
#     return df_orig

    
## Tokenize and stem data

import nltk
from nltk.stem.porter import PorterStemmer
import nltk
from nltk.corpus import stopwords


def tokenize(text):
    stop_words = set(stopwords.words('english')) 
    word_tokens = nltk.word_tokenize(text)
    filtered_sentence = [] 
    for w in word_tokens: 
        #if w not in stop_words: 
        filtered_sentence.append(w) 
    stems = []
    for item in filtered_sentence:
        stems.append(PorterStemmer().stem(item))
    return ' '.join(word for word in stems)

def preprocessingDF(df_orig):
    df_orig['tweet'] = df_orig['rtweet'].apply(lambda x: cleanData(x))
    df_orig['tweet'],df_orig['NumLink'] = zip(*df_orig['tweet'].map(removeHyperLink))
    df_orig['hashtags'] = df_orig['tweet'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
    df_orig['tweet'],df_orig['spCount'] =  zip(*df_orig['tweet'].map(removeSpChar))
    df_orig['wdCount'] = df_orig['tweet'].apply(lambda x: len(str(x).split(" ")))
    df_orig['chCount'] = df_orig['tweet'].str.len()
    df_orig['isCap'] = df_orig['tweet'].apply(lambda x: len([x for x in x.split() if x[0].isupper()]))
    df_orig['isRT'] = df_orig['tweet'].apply(lambda x: 1 if 'RT' in x else 0)
    df_orig['tweet'] = df_orig['tweet'].apply(lambda x: removeRT(x))
    df_orig['tweet'] = df_orig['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
    df_orig['tweet'] = df_orig['tweet'].apply(lambda x: tokenize(x))
    return df_orig


def trainRT(df_orig):
    ## Rate of RT
    #df_orig['atweet'] = df_orig.groupby("id")['tweet'].transform(lambda x: ' '.join(x))
    df_orig['TotTw'] = df_orig.groupby("id")['tweet'].transform('count')
    df_orig['RTrate'] = df_orig.groupby("id")['isRT'].transform('sum')
    df_orig['RTrate'] = df_orig['RTrate']/df_orig['TotTw']
    return df_orig

def testRT(df_orig):
    df_orig['RTrate'] = df_orig['isRT']
    return df_orig

def dropFeatures(df_orig):
    return df_orig.drop(columns=["NumLink","spCount","hashtags","chCount","isCap","isRT","wdCount"])


In [2]:
df_original_tweet = pd.read_table('../data/train_tweets.txt')
#df_original_tweet = preprocessing(df_original_tweet)

#df_train_tweet = normalizeDF(df_train_tweet)

/Users/saransh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [3]:
#df_original_tweet

# Model Training

In [6]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

df_tweet = df_original_tweet.iloc[:, 1:].values
y = df_original_tweet.iloc[:,0].values

encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# encoded_Y
sentences_train, sentences_test, y_train, y_test = train_test_split(df_tweet, encoded_Y, test_size=0.20,random_state=500)



In [7]:
df_train = pd.DataFrame(sentences_train,columns=["rtweet"])
df_train = preprocessingDF(df_train)



In [8]:

wdCount_scaler = preprocessing.MinMaxScaler()
wdCount = df_train['wdCount'].values #returns a numpy array

wdCount = wdCount.reshape(-1, 1)

wdCount_scaler.fit(wdCount)
wdCount_scaled = wdCount_scaler.transform(wdCount)
df_train['wdCount'] = pd.DataFrame(wdCount_scaled)



chCount_scaler = preprocessing.MinMaxScaler()
chCount = df_train['chCount'].values #returns a numpy array
chCount = chCount.reshape(-1, 1)

chCount_scaler.fit(chCount)
chCount_scaled = chCount_scaler.transform(chCount)
df_train['chCount'] = pd.DataFrame(chCount_scaled)




In [9]:
#df_train = dropFeatures(df_train)


In [10]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(df_train['tweet'].values)

X_bow_train = vectorizer.transform(df_train['tweet'].values)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_vecs = tfidf.fit(df_train['tweet'].values)

X_tfidf_train = tfidf_vecs.transform(df_train['tweet'].values)
X_tfidf_train





<262555x92911 sparse matrix of type '<class 'numpy.float64'>'
	with 2896777 stored elements in Compressed Sparse Row format>

In [12]:
from sklearn.decomposition import TruncatedSVD

svd_tfidf = TruncatedSVD(n_components=1000)
svd_tfidf.fit(X_tfidf_train)

matrix_tfidf_train_lowrank = svd_tfidf.transform(X_tfidf_train)

#matrix_tfidf_train_lowrank

In [13]:
df_tfidf_train = pd.DataFrame(matrix_tfidf_train_lowrank)


In [14]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=1000)
svd.fit(X_bow_train)

matrix_train_lowrank = svd.transform(X_bow_train)

df_bow_train = pd.DataFrame(matrix_train_lowrank)
combDF_bog_train = pd.concat([df_train.drop(columns=["rtweet","tweet"]), df_bow_train], axis=1)
#combDF_bog_train


In [15]:
combDF_train = pd.concat([combDF_bog_train, df_tfidf_train], axis=1)
#combDF_train

In [16]:
combDF_train.shape

(262555, 2007)

In [17]:
# ## To store


# import pickle

# def storeModel(model,filename):
#     pickle.dump(model, open(filename, 'wb'))
    
# SVD = "SVD_1000_v2"
# storeModel(svd,SVD)


In [18]:
####
## Preparing testing data
#####

df_test = pd.DataFrame(sentences_test,columns=["rtweet"])
df_test = preprocessingDF(df_test)
#df_test = dropFeatures(df_test)



In [19]:

# normalizing test data


wdCount_test = df_test['wdCount'].values #returns a numpy array
wdCount_test = wdCount_test.reshape(-1, 1)
wdCount_scaled_test = wdCount_scaler.transform(wdCount_test)
df_test['wdCount'] = pd.DataFrame(wdCount_scaled_test)



chCount_test = df_test['chCount'].values #returns a numpy array
chCount_test = chCount_test.reshape(-1, 1)

chCount_scaled_test = chCount_scaler.transform(chCount_test)
df_test['chCount'] = pd.DataFrame(chCount_scaled_test)


#df_test


In [20]:
X_bow_test = vectorizer.transform(df_test['tweet'].values)
matrix_test_lowrank = svd.transform(X_bow_test)

df_bow_test = pd.DataFrame(matrix_test_lowrank)
combDF_test = pd.concat([df_test.drop(columns=["rtweet","tweet"]), df_bow_test], axis=1)
#combDF_test


In [21]:
X_tfidf_test = tfidf_vecs.transform(df_test['tweet'].values)


matrix_tfidf_test_lowrank = svd_tfidf.transform(X_tfidf_test)

df_tfidf_test = pd.DataFrame(matrix_tfidf_test_lowrank)
combDF_test_total = pd.concat([combDF_test, df_tfidf_test], axis=1)
combDF_test_total




,NumLink,hashtags,spCount,wdCount,chCount,isCap,isRT,0,1,2,...,990,991,992,993,994,995,996,997,998,999
0,0,0,1,0.004773,0.007016,0,0,0.192994,-0.034884,0.099322,...,0.002936,0.000831,0.001480,0.001998,-0.000662,0.001764,0.000816,-0.000357,-0.000204,0.002191
1,1,0,0,0.033413,0.042095,5,0,0.517067,0.093948,0.278472,...,-0.047708,-0.025545,0.006834,0.046583,-0.000862,0.030476,-0.009843,0.005496,0.022838,0.008453
2,1,0,2,0.026253,0.028064,7,0,1.558949,-1.392155,-0.359748,...,0.022637,-0.017494,0.010043,0.014480,0.006798,-0.009007,0.000924,-0.001123,-0.011042,0.011405
3,1,0,3,0.040573,0.041160,9,0,0.737812,0.206194,0.737023,...,0.002685,0.007097,-0.002027,-0.015345,0.002407,-0.010998,-0.006199,0.000233,0.000223,-0.005848
4,0,0,0,0.035800,0.040225,3,0,1.853763,0.415569,0.212842,...,0.004941,-0.004954,0.002754,0.000551,0.002863,-0.000790,-0.002180,0.000387,-0.003505,0.000541
5,0,0,3,0.054893,0.051450,7,0,2.521867,1.474270,0.239612,...,-0.009460,0.007135,0.027583,-0.003945,-0.019125,-0.001194,-0.005433,-0.018909,0.006435,0.013304
6,0,0,1,0.019093,0.022919,8,0,0.368952,0.384042,0.686889,...,-0.002683,0.000581,-0.000716,-0.002572,0.000543,-0.002665,-0.005379,-0.001408,-0.001255,0.000553
7,0,0,1,0.014320,0.020112,0,0,0.107845,0.049721,0.083212,...,0.001302,-0.001838,-0.000244,-0.002563,-0.014278,-0.004763,0.008917,0.000227,0.016105,0.001629
8,0,0,1,0.038186,0.038354,0,0,1.023509,-0.041828,0.649776,...,-0.015315,-0.006243,0.003596,0.008603,-0.010224,0.015707,-0.010409,-0.003530,0.005275,0.001253
9,0,0,3,0.028640,0.032741,0,0,0.346484,0.120026,0.339969,...,-0.000388,-0.003235,0.000079,-0.004406,-0.001949,0.007234,-0.003761,-0.000397,0.005012,0.005738


In [22]:
combDF_test_total.shape

(65639, 2007)

In [23]:
X_train = combDF_train.values
X_test = combDF_test_total.values

input_dim = X_train.shape[1]



In [29]:
from keras.layers import Dropout
from keras.constraints import maxnorm
from keras.optimizers import SGD

model = Sequential()
# model.add(Dropout(0.2, input_shape=(input_dim,)))
# model.add(Dense(1000, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Dense(100, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Dense(30, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Dense(250, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(3)))

model.add(Dense(1000, input_dim=input_dim, activation='relu')) # input dimension = dimension of festure vector
model.add(Dense(100, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(9293, activation='softmax')) # output layer = no. of classes

#sgd = SGD(lr=0.1, momentum=0.9, decay=0.0, nesterov=False)

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.metrics_names)

['loss', 'acc']


In [30]:
batch_size = 1000
epochs = 20
history = model.fit(X_train,y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_test,y_test))



Train on 262555 samples, validate on 65639 samples
Epoch 1/20


KeyboardInterrupt: 

In [27]:
batch_size = 500
epochs = 30
history = model.fit(X_train,y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_test,y_test))



Train on 262555 samples, validate on 65639 samples
Epoch 1/30
262555/262555 [==============================] - 134s 511us/step - loss: 7.3421 - acc: 0.0352 - val_loss: 7.7090 - val_acc: 0.0346
Epoch 2/30
262555/262555 [==============================] - 122s 465us/step - loss: 7.2533 - acc: 0.0394 - val_loss: 7.6215 - val_acc: 0.0384
Epoch 3/30
262555/262555 [==============================] - 127s 484us/step - loss: 7.1639 - acc: 0.0425 - val_loss: 7.5385 - val_acc: 0.0461
Epoch 4/30
262555/262555 [==============================] - 129s 490us/step - loss: 7.0695 - acc: 0.0464 - val_loss: 7.6012 - val_acc: 0.0446
Epoch 5/30
262555/262555 [==============================] - 118s 449us/step - loss: 6.9885 - acc: 0.0498 - val_loss: 7.6795 - val_acc: 0.0427
Epoch 6/30
262555/262555 [==============================] - 116s 441us/step - loss: 6.9174 - acc: 0.0526 - val_loss: 7.6781 - val_acc: 0.0447
Epoch 7/30
262555/262555 [==============================] - 120s 458us/step - loss: 6.8561 - acc:

In [28]:
# batch_size = 500
# epochs = 300
# history = model.fit(X_train,y_train,
#                     batch_size=batch_size,
#                     epochs=epochs,
#                     verbose=1,
#                     validation_data=(X_test,y_test))



Train on 262555 samples, validate on 65639 samples
Epoch 1/300
262555/262555 [==============================] - 134s 510us/step - loss: 5.8561 - acc: 0.0973 - val_loss: 8.0946 - val_acc: 0.0657
Epoch 2/300
262555/262555 [==============================] - 121s 462us/step - loss: 5.8329 - acc: 0.0988 - val_loss: 8.0644 - val_acc: 0.0663
Epoch 3/300
262555/262555 [==============================] - 125s 475us/step - loss: 5.8043 - acc: 0.0995 - val_loss: 8.1641 - val_acc: 0.0621
Epoch 4/300
262555/262555 [==============================] - 119s 454us/step - loss: 5.7882 - acc: 0.1004 - val_loss: 8.1390 - val_acc: 0.0667
Epoch 5/300
262555/262555 [==============================] - 122s 466us/step - loss: 5.7546 - acc: 0.1034 - val_loss: 8.1014 - val_acc: 0.0661
Epoch 6/300
262555/262555 [==============================] - 120s 456us/step - loss: 5.7401 - acc: 0.1035 - val_loss: 8.2135 - val_acc: 0.0587
Epoch 7/300
262555/262555 [==============================] - 124s 474us/step - loss: 5.7109

262555/262555 [==============================] - 127s 482us/step - loss: 5.1166 - acc: 0.1499 - val_loss: 8.9225 - val_acc: 0.0601
Epoch 58/300
262555/262555 [==============================] - 126s 480us/step - loss: 5.0975 - acc: 0.1524 - val_loss: 8.9172 - val_acc: 0.0645
Epoch 59/300
262555/262555 [==============================] - 127s 484us/step - loss: 5.1033 - acc: 0.1511 - val_loss: 8.7876 - val_acc: 0.0692
Epoch 60/300
262555/262555 [==============================] - 131s 498us/step - loss: 5.1040 - acc: 0.1517 - val_loss: 8.9001 - val_acc: 0.0620
Epoch 61/300
262555/262555 [==============================] - 129s 491us/step - loss: 5.1066 - acc: 0.1510 - val_loss: 9.0192 - val_acc: 0.0599
Epoch 62/300
262555/262555 [==============================] - 121s 462us/step - loss: 5.1035 - acc: 0.1519 - val_loss: 9.0566 - val_acc: 0.0546
Epoch 63/300
262555/262555 [==============================] - 126s 479us/step - loss: 5.1163 - acc: 0.1501 - val_loss: 8.7620 - val_acc: 0.0668
Epoch

262555/262555 [==============================] - 121s 461us/step - loss: 4.8773 - acc: 0.1748 - val_loss: 9.1138 - val_acc: 0.0641
Epoch 114/300
262555/262555 [==============================] - 124s 473us/step - loss: 4.8687 - acc: 0.1745 - val_loss: 9.2918 - val_acc: 0.0626
Epoch 115/300
262555/262555 [==============================] - 121s 460us/step - loss: 4.8764 - acc: 0.1730 - val_loss: 9.1799 - val_acc: 0.0678E
Epoch 116/300
262555/262555 [==============================] - 124s 471us/step - loss: 4.8697 - acc: 0.1751 - val_loss: 9.1246 - val_acc: 0.0677
Epoch 117/300
262555/262555 [==============================] - 122s 466us/step - loss: 4.8609 - acc: 0.1760 - val_loss: 9.0827 - val_acc: 0.0660
Epoch 118/300
262555/262555 [==============================] - 132s 504us/step - loss: 4.8509 - acc: 0.1767 - val_loss: 9.1891 - val_acc: 0.0660
Epoch 119/300
262555/262555 [==============================] - 122s 466us/step - loss: 4.8616 - acc: 0.1760 - val_loss: 9.2057 - val_acc: 0.070

262555/262555 [==============================] - 122s 463us/step - loss: 4.7402 - acc: 0.1885 - val_loss: 9.3979 - val_acc: 0.0646
Epoch 169/300
262555/262555 [==============================] - 127s 483us/step - loss: 4.7383 - acc: 0.1885 - val_loss: 9.3646 - val_acc: 0.0672
Epoch 170/300
262555/262555 [==============================] - 126s 481us/step - loss: 4.7640 - acc: 0.1862 - val_loss: 9.3424 - val_acc: 0.0643
Epoch 171/300
262555/262555 [==============================] - 125s 478us/step - loss: 4.7181 - acc: 0.1905 - val_loss: 9.5788 - val_acc: 0.0625
Epoch 172/300
262555/262555 [==============================] - 121s 461us/step - loss: 4.7333 - acc: 0.1902 - val_loss: 9.4773 - val_acc: 0.0622
Epoch 173/300
262555/262555 [==============================] - 123s 469us/step - loss: 4.7253 - acc: 0.1899 - val_loss: 9.4509 - val_acc: 0.0661
Epoch 174/300
262555/262555 [==============================] - 125s 475us/step - loss: 4.7328 - acc: 0.1897 - val_loss: 9.3645 - val_acc: 0.0666

262555/262555 [==============================] - 115s 440us/step - loss: 4.7026 - acc: 0.1939 - val_loss: 9.5887 - val_acc: 0.0613
Epoch 224/300
262555/262555 [==============================] - 121s 460us/step - loss: 4.6743 - acc: 0.1964 - val_loss: 9.4713 - val_acc: 0.0614
Epoch 225/300
262555/262555 [==============================] - 116s 441us/step - loss: 4.6450 - acc: 0.2002 - val_loss: 9.7395 - val_acc: 0.0590
Epoch 226/300
262555/262555 [==============================] - 118s 450us/step - loss: 4.6676 - acc: 0.1975 - val_loss: 9.6600 - val_acc: 0.0598 - loss: 4.6656 - acc - ETA: 0s - loss: 4.6676 - acc: 0.19 - ETA: 0s - loss: 4.6675 - acc: 0.1
Epoch 227/300
262555/262555 [==============================] - 117s 444us/step - loss: 4.6681 - acc: 0.1972 - val_loss: 9.8177 - val_acc: 0.05290 - ETA: 9s - loss: 4.6551 - acc: 0. - ETA: 8s - loss: 4.6555 - acc: 0.1
Epoch 228/300
262555/262555 [==============================] - 117s 445us/step - loss: 4.6897 - acc: 0.1955 - val_loss: 9.4

KeyboardInterrupt: 

In [ ]:
#train_histt = model.fit(X_train, y_train, epochs=100, batch_size=256, verbose=1)


In [ ]:
import matplotlib.pyplot as plt
#plt.plot(history.history['loss'])
plt.plot(history.history['acc'])
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.show()

In [ ]:
scores = model.evaluate(X_test, y_test, batch_size=32)
print('Test loss=%s'% scores[0])
print('Test accuracy=%s'% scores[1])

In [ ]:
# ANN_MODEL = "ANN_MODEL_v4"
# VECTORIZER = "Vectorizer_v4"
# ENCODER = "ENCODER_v4"


In [ ]:

# storeModel(model,ANN_MODEL)
# storeModel(vectorizer,VECTORIZER)
# storeModel(encoder,ENCODER)


In [ ]:
# import pickle

# def loadModel(filename):
#     loaded_model = pickle.load(open(filename, 'rb'))
#     return loaded_model

# vectorizer = loadModel(VECTORIZER)
# encoder = loadModel(ENCODER)
# model = loadModel(ANN_MODEL)
# svd = loadModel(SVD)

# Testing

In [ ]:
def readFile(path):
    with open(path) as f:
        lineList = f.readlines()
    return lineList

In [ ]:
test_data = readFile('../data/test_tweets_unlabeled.txt')
df_test = pd.DataFrame(test_data,columns = ['rtweet'])


In [ ]:
df_predict = preprocessingDF(df_test)
#df_test = dropFeatures(df_test)


In [ ]:

# normalizing test data


wdCount_predict = df_predict['wdCount'].values #returns a numpy array
wdCount_predict = wdCount_predict.reshape(-1, 1)
wdCount_scaled_predict = wdCount_scaler.transform(wdCount_predict)
df_predict['wdCount'] = pd.DataFrame(wdCount_scaled_predict)



chCount_predict = df_predict['chCount'].values #returns a numpy array
chCount_predict = chCount_predict.reshape(-1, 1)

chCount_scaled_predict = chCount_scaler.transform(chCount_predict)
df_predict['chCount'] = pd.DataFrame(chCount_scaled_predict)


#df_predict


In [ ]:
X_bow_predict = vectorizer.transform(df_predict['tweet'].values)
matrix_predict_lowrank = svd.transform(X_bow_predict)

df_bow_predict = pd.DataFrame(matrix_predict_lowrank)
combDF_predict = pd.concat([df_predict.drop(columns=["rtweet","tweet"]), df_bow_predict], axis=1)
combDF_predict


In [ ]:
X_tfidf_predict = tfidf_vecs.transform(df_predict['tweet'].values)


matrix_tfidf_predict_lowrank = svd_tfidf.transform(X_tfidf_predict)

df_tfidf_predict = pd.DataFrame(matrix_tfidf_predict_lowrank)
combDF_predict_total = pd.concat([combDF_predict, df_tfidf_predict], axis=1)
combDF_predict_total




In [ ]:
# test_sentences  = vectorizer.transform(df_test['tweet'])
# matrix_test_lowrank = svd.transform(test_sentences)
# count_test_df = pd.DataFrame(matrix_test_lowrank)

# df_test

In [ ]:
combDF_predict_total.shape

In [ ]:

# ## Concatenate 
# df2_test = df_test.drop(columns=["rtweet","tweet"])
# combined_test_df = pd.concat([df2_test, count_test_df], axis=1)
# combined_test_df


In [ ]:
X_testdata = combDF_predict_total.iloc[:, 0:].values

y_testdata = model.predict_classes(X_testdata)
y_output = encoder.inverse_transform(y_testdata)


result = []
for i in range(0,len(y_output)):
    result.append(y_output[i])
print(len(result))
resDF = pd.DataFrame(result)

In [ ]:
resDF.to_csv("../features/unlabelled_prediction_v18.csv")

In [ ]:
#!kaggle competitions submit -c whodunnit -f submission.csv -m "Message"

# LSTM model

In [ ]:
# modelLSTM = Sequential()
# modelLSTM.add(Embedding(MAX_FEATURES, 128))
# modelLSTM.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2,
#                activation='tanh', return_sequences=True))
# modelLSTM.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2, activation='tanh'))
# modelLSTM.add(Dense(number_classes, activation='sigmoid'))
# modelLSTM.compile(loss='categorical_crossentropy', optimizer = 'rmsprop',
#               metrics=['accuracy'])